In [1]:
data_folder_path = 'data/'

In [6]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.chains import LLMChain

In [3]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import  pipeline, AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=500)
hf = HuggingFacePipeline(pipeline=pipe)

/Users/jskaf/miniconda3/envs/llmrag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:26<00:00,  8.80s/it]


In [4]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import  pipeline, T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10)
hf = HuggingFacePipeline(pipeline=pipe)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
question = "What is the Capital of France? "

template = """Question: {question}"""

prompt = PromptTemplate.from_template(template)
llm_chain = LLMChain(prompt=prompt, llm=hf)
print(llm_chain.run(question))

/Users/jskaf/miniconda3/envs/llmrag/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [4]:
from langchain.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "Say I am sexy"

print(chain.invoke({"question": question}))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain

In [ ]:
embeddings = HuggingFaceEmbeddings()

In [ ]:
import pandas as pd
df = pd.read_json(data_folder_path + "airbus_helicopters_train_set.json")
df = df.T
df

,original_text,reference_summary,uid
train_sum01,These general Standard Conditions of Sale appl...,These terms and conditions apply as soon as th...,train_sum01
train_sum010,Each Party represents to the other as at the d...,Each Party represents that the other is not a ...,train_sum010
train_sum0100,"All living, travelling and accommodation expen...","Expenses relating to the travelling, living an...",train_sum0100
train_sum0101,"Unless otherwise specified in the Contract, th...","Unless otherwise specified in the Contract, th...",train_sum0101
train_sum0102,Reasonable insurance coverage of risks arising...,Reasonable insurance coverage of risks arising...,train_sum0102
...,...,...,...
train_sum095,No term or provision hereof will be considered...,"No term, provision or breach shall be waived o...",train_sum095
train_sum096,Any variation or modification of the Contract ...,Any modification to the contract shall be put ...,train_sum096
train_sum097,The relationship between the Parties is solely...,No joint venture or partnership is intended no...,train_sum097
train_sum098,"The Customer shall not be entitled, without th...",Unless the Seller agrees to it through writing...,train_sum098


In [ ]:
indices_to_remove = ['train_sum0279', 'train_sum0317']

# Remove rows based on specified indices
df = df.drop(indices_to_remove, axis=0)
df

,original_text,reference_summary,uid
train_sum01,These general Standard Conditions of Sale appl...,These terms and conditions apply as soon as th...,train_sum01
train_sum010,Each Party represents to the other as at the d...,Each Party represents that the other is not a ...,train_sum010
train_sum0100,"All living, travelling and accommodation expen...","Expenses relating to the travelling, living an...",train_sum0100
train_sum0101,"Unless otherwise specified in the Contract, th...","Unless otherwise specified in the Contract, th...",train_sum0101
train_sum0102,Reasonable insurance coverage of risks arising...,Reasonable insurance coverage of risks arising...,train_sum0102
...,...,...,...
train_sum095,No term or provision hereof will be considered...,"No term, provision or breach shall be waived o...",train_sum095
train_sum096,Any variation or modification of the Contract ...,Any modification to the contract shall be put ...,train_sum096
train_sum097,The relationship between the Parties is solely...,No joint venture or partnership is intended no...,train_sum097
train_sum098,"The Customer shall not be entitled, without th...",Unless the Seller agrees to it through writing...,train_sum098


In [ ]:
df_reset = df.reset_index()
df_reset

,index,original_text,reference_summary,uid
0,train_sum01,These general Standard Conditions of Sale appl...,These terms and conditions apply as soon as th...,train_sum01
1,train_sum010,Each Party represents to the other as at the d...,Each Party represents that the other is not a ...,train_sum010
2,train_sum0100,"All living, travelling and accommodation expen...","Expenses relating to the travelling, living an...",train_sum0100
3,train_sum0101,"Unless otherwise specified in the Contract, th...","Unless otherwise specified in the Contract, th...",train_sum0101
4,train_sum0102,Reasonable insurance coverage of risks arising...,Reasonable insurance coverage of risks arising...,train_sum0102
...,...,...,...,...
406,train_sum095,No term or provision hereof will be considered...,"No term, provision or breach shall be waived o...",train_sum095
407,train_sum096,Any variation or modification of the Contract ...,Any modification to the contract shall be put ...,train_sum096
408,train_sum097,The relationship between the Parties is solely...,No joint venture or partnership is intended no...,train_sum097
409,train_sum098,"The Customer shall not be entitled, without th...",Unless the Seller agrees to it through writing...,train_sum098


In [ ]:
df_reset = df_reset.drop(["index","uid"], axis = 1)

In [ ]:
df_reset

,original_text,reference_summary
0,These general Standard Conditions of Sale appl...,These terms and conditions apply as soon as th...
1,Each Party represents to the other as at the d...,Each Party represents that the other is not a ...
2,"All living, travelling and accommodation expen...","Expenses relating to the travelling, living an..."
3,"Unless otherwise specified in the Contract, th...","Unless otherwise specified in the Contract, th..."
4,Reasonable insurance coverage of risks arising...,Reasonable insurance coverage of risks arising...
...,...,...
406,No term or provision hereof will be considered...,"No term, provision or breach shall be waived o..."
407,Any variation or modification of the Contract ...,Any modification to the contract shall be put ...
408,The relationship between the Parties is solely...,No joint venture or partnership is intended no...
409,"The Customer shall not be entitled, without th...",Unless the Seller agrees to it through writing...


In [ ]:
df_reset.to_csv(data_folder_path + "airbus_helicopters_train_set.csv")

In [ ]:
df_test = pd.read_csv(data_folder_path + "airbus_helicopters_train_set.csv")
df_test

,Unnamed: 0,original_text,reference_summary
0,0,These general Standard Conditions of Sale appl...,These terms and conditions apply as soon as th...
1,1,Each Party represents to the other as at the d...,Each Party represents that the other is not a ...
2,2,"All living, travelling and accommodation expen...","Expenses relating to the travelling, living an..."
3,3,"Unless otherwise specified in the Contract, th...","Unless otherwise specified in the Contract, th..."
4,4,Reasonable insurance coverage of risks arising...,Reasonable insurance coverage of risks arising...
...,...,...,...
406,406,No term or provision hereof will be considered...,"No term, provision or breach shall be waived o..."
407,407,Any variation or modification of the Contract ...,Any modification to the contract shall be put ...
408,408,The relationship between the Parties is solely...,No joint venture or partnership is intended no...
409,409,"The Customer shall not be entitled, without th...",Unless the Seller agrees to it through writing...


In [ ]:
def create_db_data() -> FAISS:
    loader = CSVLoader(file_path=data_folder_path + "airbus_helicopters_train_set.csv")
    transcript = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(transcript)
    db = FAISS.from_documents(docs, embeddings)
    return db


create_db_data()


In [ ]:
import re

def get_response_from_query(db, summary, original_text,  k=1):
    global hf

    docs = db.similarity_search(summary, k=k)
    docs_page_content = " ".join([d.page_content for d in docs])
    # Use regular expression to remove ": {number}"
    cleaned_text = re.sub(r': \d{1,3}', '', docs_page_content)

    # Remove leading and trailing whitespaces
    cleaned_text = cleaned_text.strip()

    print(cleaned_text)

    llm = hf

    # # template = """Question: {question} : {original_text}. You can use this {summary} to answer the question."""
    # template = """Question: {question} : {original_text}."""
    # prompt = PromptTemplate.from_template(template)
    # question = "Can you summarize this text?"
    # chain = LLMChain(llm=llm, prompt=prompt)
    # response = chain.invoke({"question": question, "original_text": original_text, "summary": summary})
    return response, docs

In [ ]:
original_text, summary = df_test["original_text"].iloc[4], df_test["reference_summary"].iloc[4]
summary

'Reasonable insurance coverage of risks arising from the Training Services are included in the Seller’s insurance policy and the Customer may request the certificate to the Seller.'

In [ ]:
db = create_db_data()
summary, original_text = df_test["original_text"].iloc[4], df_test["reference_summary"].iloc[4]
response, docs = get_response_from_query(db, summary, original_text)

original_text: Reasonable insurance coverage of risks arising from the Training Services will be included in the Seller’s insurance policy. The Seller may provide the certificate to the Customer at its request.
reference_summary: Reasonable insurance coverage of risks arising from the Training Services are included in the Seller’s insurance policy and the Customer may request the certificate to the Seller.


In [ ]:
response

{'question': 'Can you summarize this text?',
 'original_text': 'Reasonable insurance coverage of risks arising from the Training Services are included in the Seller’s insurance policy and the Customer may request the certificate to the Seller.',
 'summary': 'Reasonable insurance coverage of risks arising from the Training Services will be included in the Seller’s insurance policy. The Seller may provide the certificate to the Customer at its request.',
 'text': ': The certificate is a certificate of liability for the alleged breach of contract and the alleged breach of contract.'}